<a href="https://colab.research.google.com/github/Rafi-ur-Rashid/Audio-News-Classification/blob/main/Tranfer_Learning_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import os
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import pickle
from os import path
from keras.callbacks import ModelCheckpoint

import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
RANDOM_SEED=2245
from numpy.random import seed
seed(RANDOM_SEED)


In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
corpus_path=r"C:\\ML\\audio_news\\corpus\\"
preprocess_path=r"C:\\ML\\audio_news\\preprocessed-shuffled\\"
mfcc_path=preprocess_path+r"mfccs\\"
lables_path=preprocess_path+r"lables\\"
mel_path=preprocess_path+r"mel_specs\\"
weights_dir="C:\\ML\\audio_news\\weights\\"



In [ ]:
#drive
mfcc_path="/content/drive/My Drive/Colab Notebooks/Audio classification/mfccs/"              
lables_path="/content/drive/My Drive/Colab Notebooks/Audio classification/lables/"  

In [ ]:
X_train=pickle.load( open(mfcc_path+"train_13_2048_512.pkl",'rb'))
y_train=pickle.load( open(lables_path+"train.pkl",'rb'))
X_validation=pickle.load( open(mfcc_path+"val_13_2048_512.pkl",'rb'))
y_validation=pickle.load( open(lables_path+"val.pkl",'rb'))
#X_test=pickle.load( open(mfcc_path+"test_13_2048_512.pkl",'rb'))
#y_test=pickle.load( open(lables_path+"test.pkl",'rb'))


X_train=np.broadcast_to(X_train[..., np.newaxis], (X_train.shape[0], X_train.shape[1], X_train.shape[2], 3))
X_validation=np.broadcast_to(X_validation[..., np.newaxis], (X_validation.shape[0], X_validation.shape[1], X_validation.shape[2], 3))
#X_test=np.broadcast_to(X_test[..., np.newaxis], (X_test.shape[0], X_test.shape[1], X_test.shape[2], 3))


#VGG16

In [ ]:
X_train=X_train.reshape(X_train.shape[0],int(X_train.shape[1]/4),int(X_train.shape[2]*4),X_train.shape[3])
X_validation=X_validation.reshape(X_validation.shape[0],int(X_validation.shape[1]/4),int(X_validation.shape[2]*4),X_validation.shape[3])
#X_test=X_test.reshape(X_test.shape[0],int(X_test.shape[1]/4),int(X_test.shape[2]*4),X_test.shape[3])

input_shape = (X_train.shape[1], X_train.shape[2],  X_train.shape[3])

VGG16_MODEL=tf.keras.applications.VGG16(input_shape=input_shape,
                                               include_top=False,
                                               weights='imagenet')
VGG16_MODEL.trainable=False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(5,activation='softmax')
model = tf.keras.Sequential([
  VGG16_MODEL,
  global_average_layer,
  prediction_layer
])
saved_model="vgg16_mfcc_13_2048_512.weights.hdf5"

#Resnet50

In [ ]:
X_train=X_train.reshape(X_train.shape[0],int(X_train.shape[1]/4),int(X_train.shape[2]*4),X_train.shape[3])
X_validation=X_validation.reshape(X_validation.shape[0],int(X_validation.shape[1]/4),int(X_validation.shape[2]*4),X_validation.shape[3])
#X_test=X_test.reshape(X_test.shape[0],int(X_test.shape[1]/4),int(X_test.shape[2]*4),X_test.shape[3])

input_shape = (X_train.shape[1], X_train.shape[2],  X_train.shape[3])

RESNET50_MODEL=tf.keras.applications.ResNet50(input_shape=input_shape,
                                               include_top=False,
                                               weights='imagenet')
RESNET50_MODEL.trainable=False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(5,activation='softmax')
model = tf.keras.Sequential([
  RESNET50_MODEL,
  global_average_layer,
  prediction_layer
])
saved_model="resnet50_mfcc_13_2048_512.weights.hdf5"

#Inceptionv3

In [ ]:
X_train=X_train.reshape(X_train.shape[0],int(X_train.shape[1]/17),int(X_train.shape[2]*17),X_train.shape[3])
X_validation=X_validation.reshape(X_validation.shape[0],int(X_validation.shape[1]/17),int(X_validation.shape[2]*17),X_validation.shape[3])
#X_test=X_test.reshape(X_test.shape[0],int(X_test.shape[1]/4),int(X_test.shape[2]*4),X_test.shape[3])

input_shape = (X_train.shape[1], X_train.shape[2],  X_train.shape[3])

InceptionV3_MODEL=tf.keras.applications.InceptionV3(input_shape=input_shape,
                                               include_top=False,
                                               weights='imagenet')
InceptionV3_MODEL.trainable=False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(5,activation='softmax')
model = tf.keras.Sequential([
  InceptionV3_MODEL,
  global_average_layer,
  prediction_layer
])
saved_model="inceptionv3_MODEL_mfcc_13_2048_512.weights.hdf5"

#EfficientNetB7

In [ ]:
X_train=X_train.reshape(X_train.shape[0],int(X_train.shape[1]/4),int(X_train.shape[2]*4),X_train.shape[3])
X_validation=X_validation.reshape(X_validation.shape[0],int(X_validation.shape[1]/4),int(X_validation.shape[2]*4),X_validation.shape[3])
#X_test=X_test.reshape(X_test.shape[0],int(X_test.shape[1]/4),int(X_test.shape[2]*4),X_test.shape[3])

input_shape = (X_train.shape[1], X_train.shape[2],  X_train.shape[3])

EfficientNetB7_MODEL=tf.keras.applications.EfficientNetB7(input_shape=input_shape,
                                               include_top=False,
                                               weights='imagenet')
EfficientNetB7_MODEL.trainable=False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(5,activation='softmax')
model = tf.keras.Sequential([
  EfficientNetB7_MODEL,
  global_average_layer,
  prediction_layer
])
saved_model="efficientNetB7_mfcc_13_2048_512.weights.hdf5"

In [ ]:
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
checkpoint = ModelCheckpoint(weights_dir+saved_model, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.summary()

In [ ]:

history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30, callbacks = callbacks_list, verbose=1)

In [ ]:
model_loaded=tf.keras.models.load_model(weights_dir+"cnn_3_class_stft.weights.hdf5")
#model_loaded.evaluate(X_test,y_test)

In [ ]:
model.save(model_dir+"cnn2D_stft.h5")

In [ ]:
history = model.fit(X_train_reshaped, y_train, 
                    validation_split=0.1,
                    epochs=60,
                    verbose=1,
                    callbacks=[checkpoint])

In [ ]:
X_test_reshaped=np.reshape(X_test,newshape=(X_test.shape[0],X_test.shape[1],X_test.shape[2],1))
pickle.dump(X_test, open(test_dir+"X_test_stffs_cnn2D.pkl",'wb'))
pickle.dump(y_test, open(test_dir+"y_test_stffs_cnn2D.pkl",'wb'))


In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()